In [1]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
def get_params_to_update(net, feature_extract):
    params_to_update = net.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in net.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in net.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    return params_to_update

def make_alexnet(out_features, feature_extract = False):
    net = models.alexnet(pretrained=True)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs = net.classifier[6].in_features
    net.classifier[6] = nn.Linear(num_ftrs, out_features)
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)
    return net, optimizer

def make_mobilenet_v3(out_features, feature_extract = False):
    net = models.mobilenet_v3_small(pretrained=True)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs = net.classifier[3].in_features
    net.classifier[3] = nn.Linear(num_ftrs, out_features)
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)
    return net, optimizer

def make_inception_v3(out_features, feature_extract = False):
    net = models.inception_v3(pretrained=True)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs = net.AuxLogits.fc.in_features
    net.AuxLogits.fc = nn.Linear(num_ftrs, out_features)
    num_ftrs = net.fc.in_features
    net.fc = nn.Linear(num_ftrs, out_features)
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)
    return net, optimizer

def make_resnet(out_features, feature_extract = False):
    net = models.resnet18(pretrained=True)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs =  net.fc.in_features
    net.fc = nn.Linear(num_ftrs, len(selected_birds))
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)    
    return net, optimizer


In [2]:
import statistics
import torch.utils.data as td
import net_training
from birds_dataset import Birds270Dataset
import torch
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
import pandas as pd

# Splits a dataset randomly into a train and test set. The size of test set is 80% of the whole dataset.
# Then it trains the network
def train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size, early_stopping, is_inception):
    train_set_size = int(len(dataset)*0.8)
    test_set_size = len(dataset)-train_set_size
    train_dataset, test_dataset = td.random_split(dataset, [train_set_size, test_set_size])
    label_set = dataset.get_label_set()
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
    device = None # Wybiera CUDA jeśli jest dostępne, w przeciwnym wypadku CPU
    # device = "cpu" # Odkomentować jeżeli CUDA nie będzie działać
    results = net_training.train_and_evaluate(net, train_dataloader, test_dataloader, label_set,early_stopping = early_stopping,
                                 epochs=epochs, optimizer=optimizer, print_results=True, device=device, is_inception=is_inception)
    return results

# Creates a few networks and trains them using a random split of the dataset.
# The number of created networks is in the "repeat" argument
# It returns the final validation results for each network
def cross_validate_net(net_generator, dataset, repeat=5, epochs=20, batch_size=32, is_inception=False):
    all_results = []
    for i in range(repeat):
        print(f"Training network {i+1} ...")
        net, optimizer = net_generator()
        early_stopping = net_training.EarlyStoppingByAccuracy(patience=10)
        results = train_net_random_dataset_split(net, dataset, epochs,optimizer=optimizer,
                                                 early_stopping=early_stopping, batch_size=batch_size, is_inception=is_inception)
        all_results.append(results)
        net_training.print_final_results(results)
        del optimizer, net
        torch.cuda.empty_cache()
    return all_results
    
    
def print_validation_results(cross_validation_results):
    losses = [r["loss"] for r in cross_validation_results]
    accuracies = [r["accuracy"] for r in cross_validation_results]
    print("Losses: ", losses)
    print("Loss:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(losses), statistics.stdev(losses)))
    print("Accuracies: ", accuracies)
    print("Accuracy:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(accuracies), statistics.stdev(accuracies)))
    
def results_to_dataframe(cross_validation_results):
    normalized = pd.json_normalize(cross_validation_results)
    return normalized
    

In [3]:
dataset_dir = "../data/birds270"
selected_birds = ["ALBATROSS", 
"BALD EAGLE", 
"BARN OWL", 
"EURASIAN MAGPIE", 
"FLAMINGO",                
"MALLARD DUCK", 
"OSTRICH", 
"PEACOCK", 
"PELICAN", 
"TRUMPTER SWAN",
"MASKED BOOBY",
"EURASIAN GOLDEN ORIOLE",
"MIKADO  PHEASANT",
"HOUSE FINCH",
"ROSY FACED LOVEBIRD",
"EASTERN BLUEBIRD",
"GREY PLOVER",
"INDIAN BUSTARD",
"CUBAN TODY",
"WATTLED CURASSOW",
"BLUE HERON",
"RED WISKERED BULBUL",
"RUBY THROATED HUMMINGBIRD",
"RED HEADED WOODPECKER",
"NORTHERN JACANA",
"GLOSSY IBIS",
"ANHINGA",
"GOLDEN CHLOROPHONIA",
"KING VULTURE",
"TURQUOISE MOTMOT",
"KAKAPO",
"ELEGANT TROGON",
"WHITE TAILED TROPIC",
"NICOBAR PIGEON",
"MYNA",
"SAND MARTIN",
"BARRED PUFFBIRD",
"UMBRELLA BIRD",
"CAPUCHINBIRD",
"INDIGO BUNTING",
"RAINBOW LORIKEET",
"BIRD OF PARADISE",
"HOOPOES",
"WILSONS BIRD OF PARADISE",
"GUINEAFOWL",
"JAVA SPARROW",
"INDIAN PITTA",
"ROYAL FLYCATCHER",
"RAINBOW LORIKEET",
"CALIFORNIA CONDOR"]

#transform = transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5)) # normalizes colors to range [-1,1]
transform =  transforms.Compose([
    transforms.Normalize((0, 0, 0), (255, 255, 255)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_inception =  transforms.Compose([
    transforms.Resize([299, 299]),
    transforms.Normalize((0, 0, 0), (255, 255, 255)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = Birds270Dataset(dataset_dir,  selected_birds=selected_birds, transform=transform)

def net_generator():
    return make_resnet(out_features=len(selected_birds), feature_extract = False)

results = cross_validate_net(net_generator, dataset, repeat=5, epochs=100, batch_size=64, is_inception=False)
print_validation_results(results)

Training network 1 ...
Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.co

Epoch 4:
	train loss: 0.019201388764377405
	validation loss: 0.1475636875813563, validation accuracy: 96.84351914036266%
	Elapsed time: 0:14:37.421965
Epoch 5:
	train loss: 0.014920733955433925
	validation loss: 0.14383345990972762, validation accuracy: 97.04499664204164%
	Elapsed time: 0:11:59.708868
Epoch 6:
	train loss: 0.011521121183203465
	validation loss: 0.13916486525711716, validation accuracy: 96.64204163868368%
	Elapsed time: 0:13:09.721717
Epoch 7:
	train loss: 0.010264118709384784
	validation loss: 0.13606127900433587, validation accuracy: 96.57488247145736%
	Elapsed time: 0:10:36.834007
Epoch 8:
	train loss: 0.009009238057790955
	validation loss: 0.13566280112841372, validation accuracy: 96.57488247145736%
	Elapsed time: 0:12:01.454370
Epoch 9:
	train loss: 0.007611512812409865
	validation loss: 0.13329454832644652, validation accuracy: 96.37340496977838%
	Elapsed time: 0:12:02.031321
Epoch 10:
	train loss: 0.007277148081918695
	validation loss: 0.14007070891764278, valida

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

Epoch 0:
	train loss: 1.5334134494359206
	validation loss: 0.3940583568359078, validation accuracy: 95.83613163196776%
	Elapsed time: 0:15:10.937239
Epoch 1:
	train loss: 0.23058315399172485
	validation loss: 0.20319518657079713, validation accuracy: 97.64942914707858%
	Elapsed time: 0:12:36.078031
Epoch 2:
	train loss: 0.07153376707124766
	validation loss: 0.1398524275666279, validation accuracy: 98.05238415043654%
	Elapsed time: 0:12:04.489665
Epoch 3:
	train loss: 0.033433305602423705
	validation loss: 0.11855787918211791, validation accuracy: 97.7165883143049%
	Elapsed time: 0:12:06.337912
Epoch 4:
	train loss: 0.020941520255893173
	validation loss: 0.10860797346558965, validation accuracy: 98.11954331766286%
	Elapsed time: 0:12:03.986727
Epoch 5:
	train loss: 0.014718219870662321
	validation loss: 0.10612209862294494, validation accuracy: 98.18670248488918%
	Elapsed time: 0:12:04.424947
Epoch 6:
	train loss: 0.012690724574396154
	validation loss: 0.09706827731961928, validation ac

In [4]:
dataframe = results_to_dataframe(results)
dataframe.to_csv("../results/resnet_pretrained_50_species.csv")